In [1]:
import pandas as pd

In [2]:
import numpy as np
import ast
import re

In [4]:
pubmed = pd.read_csv("/Users/ashnadua/Desktop/IRE-project/sum_it_up_test_pubmed.csv")

In [5]:
pubmed.head()

,article,Gold Summary,Generated Summary,Rouge
0,anxiety affects quality of life in those livin...,research on the implications of anxiety in par...,the first study found that working memory perf...,"{'rouge1': array([0.23963964, 0.74301676, 0.36..."
1,small non coding rnas are transcribed into mrn...,small non coding rnas include sirna mirna pirn...,the identification of the expression signature...,"{'rouge1': array([0.15827338, 0.74157303, 0.26..."
2,ohss is a serious complication of ovulation in...,objective to evaluate the efficacy and safety ...,although all 33 patients presented with shortn...,"{'rouge1': array([0.25678119, 0.62831858, 0.36..."
3,congenital adrenal hyperplasia cah refers to a...,congenital adrenal hyperplasia is a group of a...,among the females 9 infants were identified at...,"{'rouge1': array([0.20936639, 0.608 , 0.31..."
4,type 1 diabetes t1d results from the destructi...,objective s pentoxifylline is an immunomodulat...,in summary results of the present work demonst...,"{'rouge1': array([0.2716763 , 0.71573604, 0.39..."


In [6]:
pubmed["Rouge"][0]

"{'rouge1': array([0.23963964, 0.74301676, 0.36239782]), 'rouge2': array([0.09927798, 0.30898876, 0.15027322]), 'rouge3': array([0.03797468, 0.11864407, 0.05753425])}"

In [7]:
rouge_scores = pubmed["Rouge"]

In [8]:
df = pd.DataFrame(rouge_scores)
df.head()

,Rouge
0,"{'rouge1': array([0.23963964, 0.74301676, 0.36..."
1,"{'rouge1': array([0.15827338, 0.74157303, 0.26..."
2,"{'rouge1': array([0.25678119, 0.62831858, 0.36..."
3,"{'rouge1': array([0.20936639, 0.608 , 0.31..."
4,"{'rouge1': array([0.2716763 , 0.71573604, 0.39..."


In [9]:
df['Rouge'][0]

"{'rouge1': array([0.23963964, 0.74301676, 0.36239782]), 'rouge2': array([0.09927798, 0.30898876, 0.15027322]), 'rouge3': array([0.03797468, 0.11864407, 0.05753425])}"

In [10]:
# function to parse string and convert it into a dictionary with numpy arrays
def parse_rouge(s):
    s = re.sub(r'array\(', '[', s)
    s = re.sub(r'\)', ']', s)
    return eval(s)


In [11]:
df['Rouge'] = df['Rouge'].apply(parse_rouge)

# Split the 'Rouge' column into three separate columns
df = df.join(pd.DataFrame(df.pop('Rouge').values.tolist(), index=df.index))

In [12]:
df.head()

,rouge1,rouge2,rouge3
0,"[[0.23963964, 0.74301676, 0.36239782]]","[[0.09927798, 0.30898876, 0.15027322]]","[[0.03797468, 0.11864407, 0.05753425]]"
1,"[[0.15827338, 0.74157303, 0.26086957]]","[[0.07211538, 0.34090909, 0.11904762]]","[[0.0313253, 0.14942529, 0.05179283]]"
2,"[[0.25678119, 0.62831858, 0.36456996]]","[[0.07246377, 0.17777778, 0.1029601]]","[[0.02540835, 0.0625, 0.03612903]]"
3,"[[0.20936639, 0.608, 0.31147541]]","[[0.09116022, 0.26612903, 0.13580247]]","[[0.04709141, 0.13821138, 0.07024793]]"
4,"[[0.2716763, 0.71573604, 0.39385475]]","[[0.11969112, 0.31632653, 0.17366947]]","[[0.06576402, 0.17435897, 0.09550562]]"


### Rouge score represents: Precision, Recall, and F1

In [13]:
for rouge_type in ['rouge1', 'rouge2', 'rouge3']:
    # Flatten the list
    df[rouge_type] = df[rouge_type].apply(lambda x: x[0])
    # Split the column into three separate columns
    df[[f'precision-{rouge_type}', f'recall-{rouge_type}', f'f1-{rouge_type}']] = pd.DataFrame(df[rouge_type].tolist(), index=df.index)
    df = df.drop(columns=[rouge_type])


In [14]:
df.head()

,precision-rouge1,recall-rouge1,f1-rouge1,precision-rouge2,recall-rouge2,f1-rouge2,precision-rouge3,recall-rouge3,f1-rouge3
0,0.239640,0.743017,0.362398,0.099278,0.308989,0.150273,0.037975,0.118644,0.057534
1,0.158273,0.741573,0.260870,0.072115,0.340909,0.119048,0.031325,0.149425,0.051793
2,0.256781,0.628319,0.364570,0.072464,0.177778,0.102960,0.025408,0.062500,0.036129
3,0.209366,0.608000,0.311475,0.091160,0.266129,0.135802,0.047091,0.138211,0.070248
4,0.271676,0.715736,0.393855,0.119691,0.316327,0.173669,0.065764,0.174359,0.095506


In [15]:
average_scores = df.mean().values

In [16]:
print("Average Precision Rouge-1: ", average_scores[0])
print("Average Recall Rouge-1: ", average_scores[1])
print("Average F1 Rouge-1: ", average_scores[2])
print()
print("Average Precision Rouge-2: ", average_scores[3])
print("Average Recall Rouge-2: ", average_scores[4])
print("Average F1 Rouge-2: ", average_scores[5])
print()
print("Average Precision Rouge-3: ", average_scores[6])
print("Average Recall Rouge-3: ", average_scores[7])
print("Average F1 Rouge-3: ", average_scores[8])

Average Precision Rouge-1:  0.25008053767
Average Recall Rouge-1:  0.65793603736
Average F1 Rouge-1:  0.34882658833

Average Precision Rouge-2:  0.10070240641000001
Average Recall Rouge-2:  0.25901633590999995
Average F1 Rouge-2:  0.13901021884

Average Precision Rouge-3:  0.053793890700000006
Average Recall Rouge-3:  0.13370987443000001
Average F1 Rouge-3:  0.07317077259999999
